In [60]:
import numpy as np
import scipy
from scipy import misc
from matplotlib import pyplot, cm
from numpy.linalg import eig

def GetPointEdges(Points,SigmaDistance,EdgeRadius):
    """This function constructs a graph describing similarity of points in the given 
       array.
      \param Points An array of shape (nPoint,2) where each row provides the 
             coordinates of one of nPoint points in the plane.
      \param SigmaDistance The standard deviation of the Gaussian distribution used 
             to weigh down longer edges.
      \param EdgeRadius A positive float providing the maximal length of edges.
      \return A tuple (EdgeWeight,EdgeIndices) where EdgeWeight is an array of 
              length nEdge providing the weight of all produced edges and 
              EdgeIndices is an integer array of shape (nEdge,2) where each row 
              provides the indices of two pixels which are connected by an edge."""
    edgeWeight = []
    edgeIndices = []
    for i, pointi in enumerate(Points):
        for j, pointj in enumerate(Points):
            """if i==j:
                continue"""
            dist = (pointi[0]-pointj[0])**2
            dist += (pointi[1]-pointj[1])**2
            if (dist**.5)<=EdgeRadius:
                wij = (pointi[0]-pointj[0])**2
                wij += (pointi[1]-pointj[1])**2
                wij /= 2*(SigmaDistance**2)
                wij = np.exp(-wij)
                """wij is higher the closer the edges are. 
                It is 1 if they are at the same place and close 
                to 0 as they distance from each other. """
                edgeWeight+= [wij]
                edgeIndices+= [(i,j)]
    return (edgeWeight, edgeIndices)
    


def GetLaplacian(nVertex,EdgeWeight,EdgeIndices):
    """Constructs a matrix providing the Laplacian for the given graph. 
      \param nVertex The number of vertices in the graph (resp. pixels in the 
             image).
      \param EdgeWeight A one-dimensional array of nEdge floats providing the weight 
             for each edge.
      \param EdgeIndices An integer array of shape (nEdge,2) where each row provides 
             the vertex indices for one edge.
      \return A matrix providing the Laplacian for the given graph."""
    aff = np.zeros((nVertex, nVertex), dtype=float) # affinity
    deg = np.zeros((nVertex, nVertex), dtype=float) # degree
    for w, i in zip(EdgeWeight, EdgeIndices):
        if i[0]!=i[1]:
            aff[i] += w
        deg[i[0], i[0]] += w
    return deg-aff
    


def GetFiedlerVector(Laplacian):
    """Given the Laplacian matrix of a graph this function computes the normalized 
       Eigenvector for its second-smallest Eigenvalue (the so-called Fiedler vector) 
       and returns it."""
    val, vec = eig(Laplacian)
    eigen = [(x[0],x[1]) for x in zip(val, vec)]
    eigen = sorted(eigen, key=lambda x: x[0])
    return eigen[1][1]



if(__name__=="__main__"):
    # This list of points is to be clustered
    Points=np.asarray([(-8.097,10.680),(-3.902,8.421),(-9.711,7.372),(0.859,12.859),(4.732,11.084),(-0.594,9.147)\
                       ,(-4.224,13.585),(-9.066,11.891),(-13.181,8.663),(-12.374,3.983),(-11.406,-2.068),(-9.630,2.854),\
                       (-13.665,-6.667),(-15.521,-0.454),(-15.117,-6.587),(-11.970,-10.621),(-6.000,-12.799),(-2.853,-14.978)\
                       ,(-8.501,-10.217),(2.311,-11.670),(3.441,-14.171),(5.861,-10.137),(10.138,-6.909),(15.382,-5.215),\
                       (14.091,0.675),(11.187,3.903),(8.685,8.502),(7.879,11.649),(5.216,10.680),(11.025,6.888),(13.446,2.612),\
                       (12.962,-7.393),(8.363,-9.330),(-0.594,-0.212),(1.666,1.401),(1.424,-1.019),(-0.351,-2.552),\
                       (-2.127,0.675),(-0.271,2.128),(-4.743,-4.016)]);
    nVertex=Points.shape[0];
    
    # Construct the graph for the points
    EdgeWeight,EdgeIndices=GetPointEdges(Points,1.8,7.0);
    # Construct the Laplacian matrix for the graph
    Laplacian=GetLaplacian(nVertex,EdgeWeight,EdgeIndices);
    # Compute the Fiedler vector
    FiedlerVector=GetFiedlerVector(Laplacian);
    sortedFiedler = sorted(FiedlerVector)
    threshold = sortedFiedler[0] + sortedFiedler[39] / 2
    print(FiedlerVector[FiedlerVector>0])
    # Show the results

[  8.91277678e-01   4.16537304e-02   1.28320040e-01   1.94105257e-10
   1.43644612e-01   8.49605656e-03   2.62840413e-14   1.02291820e-13
   3.92014168e-13   2.70676935e-07   3.13473045e-09   1.39638888e-07
   5.40484052e-08   9.79363543e-07   5.67845587e-07   1.03832351e-05
   6.69334942e-07   4.72530486e-04   9.44491497e-06   1.62529809e-03
   4.44091508e-03   2.68693515e-03   4.79207855e-04]


In [15]:
[(x[0], x[1]) for x in zip(['a', 'b', 'c'], [1, 2, 3])]


AttributeError: 'zip' object has no attribute 'sort'